<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2022/omake04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML omake04

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png">


----
# Python + NumPy + Matplotlib + Pandas 入門 (2)
----

おまけ03の続編です．ベクトルや行列の計算をしたり，それを応用して最小二乗法の問題を解いたりします．


----
## NumPy でベクトルや行列を扱う

前回の続きとして，NumPy でベクトルや行列を扱う方法を説明します．

### 一次元配列・ベクトル

In [ ]:
import numpy as np  # numpy モジュールをインポートして np という名前で呼べるようにする

一次元の配列（`np.array`）の話の復習から．

In [ ]:
x = np.arange(1, 6)  # 5つの要素から成る配列
print(x)
print(x.shape) # 配列 x の shape（形）．要素数 5 のベクトル（一次元配列）であることがわかる

In [ ]:
y = np.array([1, 2, np.pi, 4, 5]) # 浮動小数点数の配列． np.pi は円周率
print(y)
print(y.shape)

上記の `x` と `y` はいずれも要素数が 5 の一次元配列です．次のような演算ができるのでした．

In [ ]:
x2 = x + 2
print(x2)

In [ ]:
z = x + y
print(z)

C言語の配列と同じように要素を取り出すこともできます．

In [ ]:
x[2]

要素番号を指定している `[ ]` の中には，スライスも書けます．

In [ ]:
y[1:4]

ここで作った変数 `x` と `y` はいずれも要素数 5 の一次元配列ですので，5次元のベクトルとみなすことができます． `@` 演算子を使うと，二つのベクトル同士の内積を計算させることができます．

In [ ]:
x @ y

$\mathbf{x} = (x_1, x_2, \dots, x_{D})$ および $\mathbf{y} = (y_1, y_2, \dots, y_{D})$ のとき，$\mathbf{x}$ と $\mathbf{y}$ の内積 $\mathbf{x}\cdot\mathbf{y}$ は

$$
\mathbf{x}\cdot\mathbf{y} = \sum_{d=1}^{D} x_dy_d
$$

です．

一方，`*`演算子を使うと，同じ要素数の配列の同じ場所の値同士の積を計算することになります．`@`演算子による内積の計算と違って結果は元の配列と同じ要素数の配列です．

In [ ]:
x * y

### 二次元配列・行列

`np.array` にリストのリストを渡すと，二次元配列（=行列）を作ることができます．


In [ ]:
A = np.array( [[0, 1, 2], [3, 4, 5]]) # 2行3列の行列
print(A)
print(A.shape) # 配列 A の shape（形）． 2x3 行列であることがわかる

`reshape` というメソッドを使うと配列の形を変えることができるので，これを利用して作る手もあります．

In [ ]:
b = np.arange(6)  # b は要素数 6 の一次元配列
print(b)
B = b.reshape((3, 2)) # B は b を (3, 2) の shape で見たもの
print(B)
print(B.shape)

ここで，`b` と `B` は見た目は別物ですが，要素は同じものです（同じ記憶領域を共有している，という言い方もできます）．そのため，次のようなことになります．

In [ ]:
b[3] = 5963  # b の一つの値を変更
print(b) 
print(B)  # B の方は変更してないが...

変数`A` が表す $2\times 3$ 行列を $A$，変数`B` が表す $3\times 2$ 行列を $B$ とおくと，これら二つの積 $AB$ は $2\times 2$ 行列となります．次のように `@` 演算子を使って行列の積が計算できます．

In [ ]:
print(A @ B)

同様に積 $BA$ も計算できます．結果は $3\times 3$行列．

In [ ]:
print(B @ A)

`A.T` は `A` の転置行列．つまり，$A^{\top}$．

In [ ]:
At = A.T
print(At, At.shape)

一次元配列の場合と同様に，`*`は要素ごとの積を求める演算子です．

In [ ]:
hoge = np.arange(4)
C = hoge.reshape((2, 2))
print(C)
print(C @ C) # 行列 C と C の積，つまり C の二乗
print(C * C) # C の要素ごとの二乗

一次元配列の場合と同様に，`[ ]`の中に要素番号を指定することで要素の値を参照したり値を代入したりできます．

In [ ]:
print(A)
print(A[1, 2]) # 0から数えて1行目，0から数えて2列目

スライスも使えます．

In [ ]:
print(A[1, :2]) # 1行目の0列目と1列目のみ取り出した配列

In [ ]:
print(A[:, 1])  # 全ての行の1列目のみ取り出した配列

In [ ]:
print(A[:, 1:])

----
## 練習問題


以下のセルを実行すると，変数 `dataX` と `dataY` の値が， ex02noteA の練習問題（自動車価格の変化を表すデータの回帰分析）の $x_n$ と $y_n$ の値をそれぞれ並べた配列になります．


In [ ]:
dataX = np.array([0, 4, 8, 20])
dataY = np.array([0, 1, -1, -2])

最小二乗法でこのデータに当てはまる直線を求めてみましょう．

以下のセルを実行すると，変数 `AA` と `bb` がそれぞれ，以下のような $2\times 2$ 行列 $A$ と 2次元ベクトル（$2\times 1$行列） $\mathbf{b}$ を表すことになります．
$$
A = \begin{pmatrix}
\displaystyle\sum_{n=1}^{N}x_n^2 & \displaystyle\sum_{n=1}^{N}x_n\\
\displaystyle\sum_{n=1}^{N}x_n & \displaystyle\sum_{n=1}^{N}1\\
\end{pmatrix} \qquad
\mathbf{b} = 
\begin{pmatrix}
\displaystyle\sum_{n=1}^{N}x_ny_n\\
\displaystyle\sum_{n=1}^{N}y_n
\end{pmatrix} 
$$

In [ ]:
XX = np.vstack((dataX, np.ones_like(dataX))).T
#print(XX)
YY = dataY[:, np.newaxis]
#print(YY)
AA = XX.T @ XX
bb = XX.T @ dataY
print(AA)
print(bb)

(1) 授業で説明しているように，最小二乗法による直線当てはめで得られる直線の傾きを $a$，切片を $b$ として，
$$
\mathbf{x} = \begin{pmatrix}
a \\ b
\end{pmatrix}
$$
とおくと，$(a, b)$ は，連立方程式
$$
A\mathbf{x} = \mathbf{b}
$$
の解です． numpy には連立方程式の解を求める関数がありますので，それを使えば，$(a, b)$ を求めることができます．以下に新しいセルを作ってやってみましょう．

上記の計算によって得られた結果が手計算で解いた結果と一致することを確認しましょう．

(2) 以下のセルを修正して実行すると，データの散布図に求めた直線を重ねて描くことができます．やってみましょう．

In [ ]:
# グラフ描画のためのモジュールをインポート
import matplotlib.pyplot as plt
# seaborn は matplotlib のグラフをより綺麗に表示させるためのモジュール 
import seaborn
seaborn.set()

# a, b を修正しましょう
a = -0.2
b = 0.5
fig, ax = plt.subplots()
ax.scatter(dataX, dataY)
ax.set_xlim(-1, 21)
xr = np.array(ax.get_xlim())
yr = a*xr+b
ax.plot(xr, yr, color='red')
plt.show()

(3) 以下のセルを実行すると，「ゴリゴリくん」のデータを変数 `dataX` と `dataY` に代入することができます．このセルに上で書いたコードをコピペして実行すると，このデータに対する直線当てはめの解も求められます．やってみましょう．

In [ ]:
# データ分析のためのモジュール
import pandas as pd

dfGori = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/gorigori.csv')
dataX = dfGori['気温']
dataY = dfGori['アイス売上数']

# ここを書いて，a, b に適切な値が代入されるようにしよう

a = 2
b = 10
fig, ax = plt.subplots()
ax.scatter(dataX, dataY)
ax.set_xlim(-5, 40)
ax.set_ylim(0, 130)
xr = np.array(ax.get_xlim())
yr = a*xr+b
ax.plot(xr, yr, color='red')
plt.show()